In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import OneCycleLR
import torchvision
import torchvision.transforms as transforms
import tqdm
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from sklearn.metrics import roc_curve, accuracy_score, f1_score, auc, precision_recall_curve
from statistics import mean

### Configuration

In [41]:
batch_size = 64
classes = ("plane", "car", "bird", "cat",
           "deer", "dog", "frog", "horse", "ship", "truck")
max_lr = 1e-3
n_epochs = 5
model_name = "CNN-2D"

### Dataset

In [13]:
transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])
train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
train_dataset_loader = torch.utils.data.DataLoader(train_dataset, 
                                                   batch_size=batch_size, 
                                                   shuffle=True)                                   

test_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
test_dataset_loader = torch.utils.data.DataLoader(test_dataset, 
                                                  batch_size=batch_size, 
                                                  shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


### Model

In [38]:
class CNN_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_1 = nn.Conv2d(3, 6, 5, stride=1, padding="same")
        self.conv_2 = nn.Conv2d(6, 16, 5, stride=1, padding="same")
        self.maxpool = nn.MaxPool2d(2, 2)
        self.linear_1 = nn.Linear(16 * 8 * 8, 120)
        self.linear_2 = nn.Linear(120, 84)
        self.linear_3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.maxpool(F.relu(self.conv_1(x)))
        x = self.maxpool(F.relu(self.conv_2(x)))
        
        x = torch.flatten(x, 1) # flatten all dimension except the first one which corresponds to batch
        
        x = F.relu(self.linear_1(x))
        x = F.relu(self.linear_2(x))
        
        x = self.linear_3(x)
        return x

In [39]:
model = CNN_Model()

### Loss and Optimizer

In [47]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
lr_scheduler = OneCycleLR(
                    optimizer=optimizer,
                    max_lr=max_lr,
                    epochs=n_epochs,
                    steps_per_epoch=len(train_dataset_loader),
                    pct_start=0.1,
                    anneal_strategy='cos',
                    div_factor=25.0,
                    final_div_factor=10000.0)
tbw = SummaryWriter() # tensorboard summary writer

### Training

In [48]:
def train():
    model.train_iter = 0
    model.val_iter = 0
    
    for epoch in range(n_epochs):
        model.train()
        for _, data in enumerate(pbar := tqdm.tqdm(train_dataset_loader, 0)):
            model.train_iter += 1 
            # get input and labels; data is a list of [(inputs, labels)]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            # loss
            loss = criterion(outputs, labels)
            # backward propagate loss
            loss.backward()
            # update parameters
            optimizer.step()
            lr_scheduler.step()

            # training_loss
            train_loss = loss.item()
            # tensorboard + logs
            tbw.add_scalar(f"{model_name}/training-loss", float(train_loss), model.train_iter)
            pbar.set_description(f"{model_name}/training-loss={train_loss}, steps={model.train_iter}, epoch={epoch+1}")
            
        validate(model, epoch)
        
    return model, validate(model, epoch)
    
def validate(model, epoch=0):
    with torch.no_grad():
        model.eval()

        results = []

        for _, data in enumerate(pbar := tqdm.tqdm(test_dataset_loader, 0)):
            # get input and labels; data is a list of [(inputs, labels)]
            inputs, labels = data

            output = model(inputs)
            # loss
            loss = criterion(output, labels)
            curr_val_loss = loss.item()
            model.val_iter += 1

            # tensorboard + logs
            tbw.add_scalar(f"{model_name}/validation-loss", float(curr_val_loss), model.val_iter)
            pbar.set_description(f"{model_name}/validation-loss={curr_val_loss}, steps={model.val_iter}, epoch={epoch+1}")

            # to get probabilities of the output
            output = F.softmax(output, dim=-1)
            result_df = pd.DataFrame(output.cpu().numpy())
            result_df["y_true"] = labels.cpu().numpy()
            results.append(result_df)
    
    return pd.concat(results, ignore_index=True)
        


In [ ]:
model, results = train()

CNN-2D/training-loss=2.3098697662353516, steps=210, epoch=1:  27%|████████████████▊                                              | 209/782 [00:46<02:12,  4.32it/s]

In [ ]:
results

In [37]:
auprcs = []
for i in range(10):
    precision, recall, _ = precision_recall_curve(y_true=results["y_true"].values, probas_pred=results[i].values, pos_label=i)
    auprc = auc(recall, precision)
    print(f"AUPRC for class {i} = {auprc}")
    auprcs.append(auprc)

macro_auprc = mean(auprcs)
print(f"Macro AUPRC = {macro_auprc}")

AUPRC for class 0 = 0.21005013117941906
AUPRC for class 1 = 0.18453593516647848
AUPRC for class 2 = 0.1310956042972487
AUPRC for class 3 = 0.13474339035829364
AUPRC for class 4 = 0.14889935264642115
AUPRC for class 5 = 0.12173970517548381
AUPRC for class 6 = 0.24230072657091292
AUPRC for class 7 = 0.20533794780987127
AUPRC for class 8 = 0.1814796143209802
AUPRC for class 9 = 0.19078316126738415
Macro AUPRC = 0.17509655687924933
